# Station Matching

The goal of this notebook is to identify stations that changed IDs. This has been known to occur for Maritime and ASOSOAWOS stations.


## Environment set-up

In [ ]:
from shapely.geometry import Point
from shapely.ops import nearest_points

import datetime
import boto3
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from io import BytesIO, StringIO

# Import qaqc stage calc functions
from QAQC_pipeline import *
# import tempfile  # Used for downloading (and then deleting) netcdfs to local drive from s3 bucket
import os

# Silence warnings
import warnings
from shapely.errors import ShapelyDeprecationWarning

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings(
    "ignore", category=ShapelyDeprecationWarning
)  # Warning is raised when creating Point object from coords. Can't figure out why.

plt.rcParams["figure.dpi"] = 300

In [ ]:
# AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client("s3")

## AWS buckets
bucket = "wecc-historical-wx"
qaqcdir = "3_qaqc_wx/"
mergedir = "4_merge_wx/"

## Step 1: Load station lists for testing

In [ ]:
### Read in ASOSAWOS stations

s3_cl = boto3.client("s3")  # for lower-level processes

asosawos = s3_cl.get_object(
    Bucket="wecc-historical-wx",
    Key="2_clean_wx/ASOSAWOS/stationlist_ASOSAWOS_cleaned.csv",
)
asosawos_list = pd.read_csv(BytesIO(asosawos["Body"].read()))

In [ ]:
valleywater = s3_cl.get_object(
    Bucket="wecc-historical-wx",
    Key="2_clean_wx/VALLEYWATER/stationlist_VALLEYWATER_cleaned.csv",
)
valleywater_list = pd.read_csv(BytesIO(valleywater["Body"].read()))

In [ ]:
maritime = s3_cl.get_object(
    Bucket="wecc-historical-wx",
    Key="2_clean_wx/MARITIME/stationlist_MARITIME_cleaned.csv",
)
maritime_list = pd.read_csv(BytesIO(maritime["Body"].read()))

## Step 2: Identify candidates for concatenation and upload to AWS

We do so by identifying stations with exactly matching latitudes and longitudes.

Some additional methods to use:
1. matching IDs, for stations in which those exist (NOT currently used)
2. stations within a certain distance of each other (I've investigated this some, but would take consideraly more time to fully develop and may not be necessary)

### Final Process

In [ ]:
# A list of networks to be checked for concatenation
target_networks = ["VALLEYWATER"]  # , "ASOSAWOS", "MARITIME"]

In [ ]:
def concatenation_check(station_list):
    """
    This function flags stations that need to be concatenated.

    Rules
    ------
        1.) Stations are flagged if they have identical latitudes and longitudes

    Parameters
    ------
        station_list: pd.DataFrame
            list of station information

    Returns
    -------
        if success:
            new_station_list: pd.DataFrame
                input station list with a flag column assigning an integer to each group of repeat latitudes and longitudes

        if failure:
            None

    """
    ##### Flag stations with identical latitudes and longitudes, then assign each group a unique integer

    # List of possible variable names for longitudes and latitudes
    lat_lon_list = ["LAT", "LON", "latitude", "longitude", "LATITUDE", "LONGITUDE", 'lat','lon']
    # Extract the latitude and longitude variable names from the input dataframe
    lat_lon_cols = [col for col in station_list.columns if col in lat_lon_list]

    # Generate column flagging duplicate latitudes and longitudes
    station_list["concat_subset"] = station_list.duplicated(
        subset=lat_lon_cols, keep=False
    )
    # within each group of identical latitudes and longitudes, assign a unique integer
    station_list["concat_subset"] = (
        station_list[station_list["concat_subset"] == True].groupby(lat_lon_cols).ngroup()
    )

    ##### Order station list by flag
    concat_station_list = station_list.sort_values("concat_subset")

    ##### Keep only flagged stations
    concat_station_list = concat_station_list[~concat_station_list["concat_subset"].isna()]

    ##### Format final list
    # Convert flags to integers - this is necessary for the final concatenation step
    concat_station_list["concat_subset"] = concat_station_list["concat_subset"].astype(
        "int32"
    )
    # Now keep only the ERA-ID and flag column
    era_id_list = ['ERA-ID','era-id']
    era_id_col = [col for col in station_list.columns if col in era_id_list]
    concat_station_list = concat_station_list[era_id_col + ["concat_subset"]]

    # Standardize ERA id to "ERA-ID" (this is specific to Valleywater stations)
    if 'era-id' in era_id_col:
        concat_station_list.rename(columns={"era-id": "ERA-ID"}, inplace=True)

    return concat_station_list

In [ ]:
def apply_concat_check(station_names_list):
    """
    This function applies the conatenation check to a list of target stations. 
    It then upload a csv containing the ERA IDs and concatenation subset ID for 
    all identified stations in a network.

    Parameters
    ------
        station__names_list: pd.DataFrame
            list of target station names

    Returns
    -------
        if success:
            uploads list of stations to be concatenated to AWS
        if failure:
            None

    """
    final_list = pd.DataFrame([])
    for station in station_names_list:

        ##### Import station list of target station
        key = "2_clean_wx/{}/stationlist_{}_cleaned.csv".format(station,station)
        bucket_name = "wecc-historical-wx"
        list_import = s3_cl.get_object(
            Bucket=bucket,
            Key=key,
        )
        station_list = pd.read_csv(BytesIO(list_import["Body"].read()))

        ##### Apply concatenation check
        concat_list = concatenation_check(station_list)

        ##### Rename the flags for each subset to <station>_<subset number>
        concat_list["concat_subset"] = station + '_' + concat_list["concat_subset"].astype(str)

        ##### Append to final list of stations to concatenate
        final_list = pd.concat([final_list,concat_list])

        ##### Upload to QAQC directory in AWS
        new_buffer = StringIO()
        final_list.to_csv(new_buffer, index = False)
        content = new_buffer.getvalue()

        s3_cl.put_object(
            Bucket = bucket_name,
            Body = content,
            Key = qaqcdir + station + "_copy" "/"+ station + "/" + station + "_concat_list_TEST.csv"
            #Key = qaqcdir + station + "/{}_concat_list_TEST.csv".format(station)
        )
        
    return None

In [ ]:
test = apply_concat_check(target_networks)

### Check that stations already indentified for concatenation are flagged

Maritime station:

- MTYC1 and MEYC1

- SMOC1 and ICAC1

In [ ]:
maritime_out

# Flagged Stations:
# MARITIME_LJAC1 <=> MARITIME_LJPC1
# MARITIME_ICAC1 <=> MARITIME_SMOC1
# MARITIME_MEYC1 <=> MARITIME_MTYC1 <=> MARITIME_MYXC1

The previously identified stations are indeed flagged. Along with an additional pair: MARITIME_LJAC1 and MARITIME_LJPC1. And a third station included with MARITIME_MEYC1 abd MARITIME_MTYC1: MARITIME_MYXC1.

### TESTING

#### using ICAO values

In [ ]:
repeat_list = asosawos_list[asosawos_list.duplicated(subset=["ICAO"], keep=False)]

# how many unique ICAO duplicates are there?
print(len(repeat_list["ICAO"].unique()))

print(repeat_list.groupby("ICAO").count().max())

In [ ]:
print(len(repeat_list["ICAO"].unique()))

#### Investigate problem station KMLF

In [ ]:
kmlf = repeat_list[repeat_list["ICAO"] == "KMLF"]

In [ ]:
kmlf[["STATION NAME", "LAT", "LON", "start_time", "end_time"]]

#### using station locations (lat, lons)

In [ ]:
# test dataframe

test = asosawos_list

In [ ]:
lat_lon_list = ["LAT", "LON", "latitude", "longitude", "LATITUDE", "LONGITUDE"]

In [ ]:
lat_lon_cols = [col for col in test.columns if col in lat_lon_list]

In [ ]:
lat_lon_cols

In [ ]:
test["concat_flag"] = asosawos_list.duplicated(subset=lat_lon_cols, keep=False)

In [ ]:
test["concat_flag"] = test[test["concat_flag"] == True].groupby(lat_lon_cols).ngroup()

In [ ]:
time_var_list = ["end_time", "end-date"]
end_time_col = [col for col in test.columns if col in time_var_list]

In [ ]:
end_time_col

In [ ]:
test = test.sort_values("concat_flag")
test = (
    test.groupby(["concat_flag"])
    .apply(lambda x: x.sort_values(end_time_col))
    .reset_index(drop=True)
)

#### Comparing ICAO identification and lat lon identification for ASOSAWOS stations

## Step 3: Carry Out Concatenation

### The Functions

In [ ]:
def _multiindex_concat_nooverlap(m_old: xr.Dataset, m_new: xr.Dataset, name: str) -> pd.DataFrame:
    """
    Formats MultiIndex, ensuring that there are no duplicate times in the time index.

    Rules
    ------
    1.) Drop duplicate times

    Parameters
    ------
    m_old: xr.Dataset
        older weather station dataset
    m_new: xr.Dataset   
        newer weather station dataset
    name: str
        newer station name
        
    Returns
    -------
    if success:
        return a pd.DataFrame with a re-formatted MultiIndex
    if failure:
        None
    """

    # combine time indices of two multiindexes
    tidx = (
        pd.concat(
            [
                pd.Series(m_old.get_level_values("time").values),
                pd.Series(m_new.get_level_values("time").values),
            ]
        )
        .reset_index()
        .drop(columns="index")
    )

    # idenitify if there are duplicate times
    tidx = tidx.rename(columns={0: "time"})
    tidx = tidx.sort_values("time").drop_duplicates(subset=["time"])

    # PULL the station name from m_new and set to the same length
    stnidx = (
        pd.Series(name, index=np.arange(len(tidx)), name="station")
        .reset_index()
        .drop(columns="index")
    )

    # combine into new df
    df_new = pd.concat([stnidx, tidx], axis=1)

    return df_new

In [ ]:
def rename_file(network: str, old_name: str, new_name: str) -> None:
    try:
        s3.copy_object(
            Bucket=bucket, CopySource=f"s3://{bucket}/3_qaqc_wx/{old_name}", Key=new_name
        )
        s3.delete_object(Bucket=bucket, Key=old_name)
        print(f"File {old_name} renamed to {new_name}")
    except Exception as e:
        print(f"Error renaming file: {e}")

In [ ]:
def _concat_export_help(
    df_concat: pd.DataFrame,
    final_concat_list: pd.DataFrame,
    network_name: str,
    attrs_new: dict,
    station_names: dict,
) -> None:
    """
    Prepares the final concatenated dataset for export by 
    - updating the attributes and 
    - converting one of the mulit-index levels to the correct datatype
    then exports the dataset to AWS

    Rules
    ------
        1.) retains the name of the newest station

    Parameters
    ------
        df_concat: pd.DataFrame
            dataframe of concatenated dataframes
        final_concat_list: pd.DataFrame

        network_name: str
            weather station network
        attrs_new: dict
            attributes of newer dataframe that was input to concatenation
        station_names: dict
            library of station names, including the single new station name and a string of all the older station names

    Returns
    -------
        if success:
            None
            exports dataset of concatenated dataframes to AWS
        if failure:
            None
    """

    ##### Rename
    

    ##### Prepare final concatenated dataset for export

    # Delete unnecessary columns and set index
    df_concat = df_concat.drop(["hour", "day", "month", "year", "date"], axis=1)
    df_to_export = df_concat.set_index(["station", "time"])

    # Convert concatenated dataframe to dataset
    ds_concat = df_to_export.to_xarray()

    # Convert datatype of station coordinate
    ds_concat.coords["station"] = ds_concat.coords["station"].astype("<U20")

    # Include past attributes
    for i in attrs_new:
        ds_concat.attrs[i] = attrs_new[i]

    # Update 'history' attribute
    timestamp = datetime.datetime.utcnow().strftime("%m-%d-%Y, %H:%M:%S")
    ds_concat.attrs["history"] = ds_concat.attrs[
        "history"
    ] + " \nstation_matching.ipynb run on {} UTC".format(timestamp)

    # Update 'comment' attribute
    ds_concat.attrs["comment"] = (
        "Intermediary data product. This data has been subjected to cleaning, QA/QC, but may not have been standardized."
    )

    # Extract old and new station names from name dictionary
    station_name_new = station_names["station_name_new"]
    station_name_old = station_names["old_stations"]

    # Add new qaqc_files_merged attribute
    ds_concat.attrs["qaqc_files_merged"] = (
        "{}, {} merged. Overlap retained from newer station data.".format(
            station_name_old,
            station_name_new  # extract old and new station names from name dictionary
        )
    )

    ##### Export
    # ! a test name is used below
    # ! the final name will be that of the newer dataframe
    export_url = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}_{}.zarr".format(
        network_name, "TEST_concat", station_name_new
    )
    print("Exporting....", export_url)
    # ds_concat.to_zarr(export_url, mode="w") ## WHEN READY TO EXPORT

    # ! output final concatenated dataset for testing
    return ds_concat 

    # return None

In [ ]:
def _overlap_concat(df_new: pd.DataFrame, df_old: pd.DataFrame) -> pd.DataFrame:
    """
    Handles the cases in which there is overlap between the two input stations

    Rules
    ------
        1.) concatenation: keep the newer station data in the time range in which both stations overlap

    Parameters
    ------
        df_new: pd.DataFrame
            weather station network
        df_old: pd.DataFrame
            weather station network

    Returns
    -------
        if success:
            df_concat: pd.DataFrame
        if failure:
            None
    """

    df_overlap = df_new[df_new["time"].isin(df_old["time"])]

    ##### Split datframes into subsets #####

    # Remove data in time overlap between old and new
    df_old_cleaned = df_old[~df_old["time"].isin(df_overlap["time"])]
    df_new_cleaned = df_new[~df_new["time"].isin(df_overlap["time"])]

    ##### Concatenate subsets #####
    df_concat = pd.concat([df_old_cleaned, df_overlap, df_new_cleaned])

    return df_concat

In [ ]:
def _df_concat(
    df_1: pd.DataFrame, df_2: pd.DataFrame, attrs_1: dict, attrs_2: dict
) -> tuple[pd.DataFrame, str, str, dict]:
    """
    Performs concatenation of input datasets, handling two cases
        1.) temporal overlap between the datasets
        2.) no temporal overlap

    Rules
    ------
        1.) concatenation: keep the newer station data in the time range in which both stations overlap

    Parameters
    ------
        df_1: pd.DataFrame
            station data
        df_2: pd.DataFrame
            dtation data
        attrs_1: list of str
            attributes of df_1
        attrs_2: list of str
            attributes of df_2

    Returns
    -------
        if success:
            df_concat: pd.DataFrame
            stn_n_to_keep: str
            stn_n_to_drop: str
            attrs_new: dict

        if failure:
            None
    """

    # determine which dataset is older
    if df_1["time"].max() < df_2["time"].max():
        # if df_1 has an earlier end tiem than df_2, then d_2 is newer
        # we also grab the name of the newer station in this step, for use later
        df_new = df_2
        attrs_new = attrs_2
        df_old = df_1

    else:
        df_new = df_1
        attrs_new = attrs_1
        df_old = df_2

    stn_n_to_keep = df_new["station"].unique()[0]
    stn_n_to_drop = df_old["station"].unique()[0]
    print(f"Station will be concatenated and saved as: {stn_n_to_keep}")

    # now set things up to determine if there is temporal overlap between df_new and df_old
    df_overlap = df_new[df_new["time"].isin(df_old["time"])]

    # If there is no overlap between the two time series, just concatenate
    if len(df_overlap) == 0:
        print("No overlap!")
        df_concat = pd.merge(df_old, df_new, how="outer")
        df_concat["station"] = stn_n_to_keep

    # If overlap exists, split into subsets and concatenate
    else:
        print("There is overlap")
        df_concat = _overlap_concat(df_old, df_new)

    return df_concat, stn_n_to_keep, stn_n_to_drop, attrs_new

In [ ]:
def _more_than_2(network_name: str, stns_to_pair: pd.DataFrame) -> tuple[pd.DataFrame, dict, dict]:
    """
    Performs pairwise concatenation on subsets of more than two stations flagged for concatenation

    Rules
    ------
        1.) concatenation: keep the newer station data in the time range in which both stations overlap

    Parameters
    ------
        network_name: string
            weather station network
        stns_to_pair: pd.DataFrame
            dataframe of the input station names

    Returns
    -------
        if success:
            df_concat: pd.DataFrame
            station_names: dict
            attrs_new: dict
        if failure:
            None
    """

    print("Concatenating the following stations:", stns_to_pair)

    # Load datasets into a list
    datasets = [
        xr.open_zarr(
            "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
                network_name, stn
            ),
            consolidated=True,
        )
        for stn in stns_to_pair['ERA-ID']
    ]

    # Sort datasets by their max 'time'
    datasets_sorted = sorted(datasets, key=lambda ds: ds['time'].max())

    # Store station names, in order from oldest to newest
    names = [ds.coords["station"].values[0] for ds in datasets_sorted]

    print('Newest station:', names[-1])

    # Setup for the while loop
    ds_1 = datasets_sorted[0]
    df_1, MultiIndex_1, attrs_1, var_attrs_1, era_qc_vars_1 = qaqc_ds_to_df(
        ds_1, verbose=False
    )
    i = 0
    end = len(datasets_sorted) -1

    while i < end:

        print('iteration:', i)

        ds_2 = datasets_sorted[i+1]
        df_2, MultiIndex_2, attrs_2, var_attrs_2, era_qc_vars_2 = qaqc_ds_to_df(
            ds_2, verbose=False
        )

        # Send to helper function for concatenation
        df_concat, stn_n_to_keep, stn_n_to_drop, attrs_new = _df_concat(
            df_1, df_2, attrs_1, attrs_2
        )

        df_1 = df_concat
        attrs_1 = attrs_new

        i += 1

    # Construct station names list, for updating attributes
    newest_station = names[-1] # Get last station name from station name list
    older_stations = ", ".join(names[:-1]) # Create a string containing all older station names
    station_names = {"station_name_new": newest_station, "old_stations": older_stations}

    print('Progressive concatenation for 2+ stations is complete.')

    new_column = [newest_station] * len(df_concat)

    df_concat['station'] = new_column

    return df_concat, station_names, attrs_new

In [ ]:
def concatenate_stations(network_name: str) -> list[str]:
    """
    Coordinates the concatenation of input datasets and exports the final concatenated dataset.
    Also returns a list of the ERA-IDs of all stations that are concatenated.

    Parameters
    ------
        network_name: string
            weather station network

    Returns
    -------
        if success:
            final_concat_list: list[str]
        if failure:
            None
    Notes
    -------
    Uses the following helper functions
        _df_concat(): concatenates two dataframes
        _overlap_concat(): used by _df_concat() to concatenate two stations with overlapping time ranges
        _more_than_2(): handles subsets with more than two stations, passing pairs to _df_concat() iteratively
        _concat_export_help(): formats and exports concatenated dataframe

    """
    # Initiate empty list, to which we will iteratively add the ERA-IDs of stations that are concatenated
    final_concat_list = []

    # Read in full concat station list
    print(network_name)
    concat_list = pd.read_csv(
        f"s3://wecc-historical-wx/3_qaqc_wx/{network_name}/concat_list_{network_name}.csv"
    )

    # Identify stns within designated network
    concat_by_network = concat_list.loc[
        concat_list.concat_subset.str.contains(network_name)
    ]

    ######### ! for testing
    concat_by_network = concat_list[concat_list["concat_subset"] == "ASOSAWOS_3"]
    # concat_by_network = concat_by_network.head(4)
    ######### ! for testing

    # For MARITIME, remove these stations becuase they're actually separate stations
    if network_name == 'MARITIME':
        unique_pair_names = unique_pair_names[1:]
        unique_pair_name = unique_pair_name[~unique_pair_name["ERA-ID"].isin['MARITIME_LJPC1','MARITIME_LJAC1']]
    else: 
        pass

    unique_pair_names = concat_by_network.concat_subset.unique()
    print(
        f"There are {len(concat_by_network)} stations to be concatenated into {len(unique_pair_names)} station pairs within {network_name}..."
    )

    print(unique_pair_names)

    # Set up pairs
    for pair in unique_pair_names:
        print(pair)
        # Pull out stations corresponding to pair name
        stns_to_pair = concat_by_network.loc[concat_by_network.concat_subset == pair]

        if len(stns_to_pair) == 2:  # 2 stations to concat together
            print("\n", stns_to_pair)

            # Import this subset of datasets and convert to dataframe
            url_1 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
                network_name, stns_to_pair.iloc[0]["ERA-ID"]
            )
            url_2 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
                network_name, stns_to_pair.iloc[1]["ERA-ID"]
            )

            print("Retrieving....", url_1)
            print("Retrieving....", url_2)
            ds_1 = xr.open_zarr(url_1)
            ds_2 = xr.open_zarr(url_2)

            # Convert to dataframes with corresponding information
            df_1, MultiIndex_1, attrs_1, var_attrs_1, era_qc_vars_1 = qaqc_ds_to_df(
                ds_1, verbose=False
            )
            df_2, MultiIndex_2, attrs_2, var_attrs_2, era_qc_vars_2 = qaqc_ds_to_df(
                ds_2, verbose=False
            )

            # Send to helper function for concatenation
            df_concat, stn_n_to_keep, stn_n_to_drop, attrs_new = _df_concat(
                df_1, df_2, attrs_1, attrs_2
            )

            # Construct dictionary of old and new station names
            station_names ={"station_name_new":stn_n_to_keep, "old_stations":stn_n_to_drop}

        else:
            # If there are more than 2 stations in the given subset, pass to _more_than_2()
            print("More than 2 stations within a subset.")
            df_concat, station_names, attrs_new = _more_than_2(
                network_name,
                stns_to_pair
            )

        # Add concatenated station names to station name list
        final_concat_list.extend(stns_to_pair["ERA-ID"].tolist())

        # Send concatenated dataframe to helper function for export
        ds_final = _concat_export_help(  #! returns ds_final for testing
            df_concat,
            final_concat_list,
            network_name,
            attrs_new,
            station_names,
        )

    print("Concatenated stations: ", final_concat_list)

    return ds_final, final_concat_list

## TEST

In [ ]:
network_name = "ASOSAWOS"
final_concat_list = concatenate_stations(network_name)

# Scraps

Here is the order of operations:

1. Read in target stations, for each concat_flag
2. Check if there is overlap in time ranges
    1. IF so:  

        split overall time range

        construct dataset by grabbing newest station for each time range subset

    
    2. ELSE:

        concatenate, with NAs in the gap


Another option: pairwise concetenation

For each subset of matching stations, first concatenate the two newest stations. Then, the next oldest, etc.


Issues to address:

1. when the time range of one station completely includes that of another in a subset (this occures a few times with ASOSAWOS)



### Concatenate pairs of stations

In [ ]:
# Lists of variables to be assigned

float64_variables = [
    "anemometer_height_m",
    "elevation",
    "lat",
    "lon",
    "pr_15min",
    "thermometer_height_m",
    "ps",
    "tas",
    "tdps",
    "pr",
    "sfcWind",
    "sfcWind_dir",
    "ps_altimeter",
    "pr_duration",
    "ps_eraqc",
    "tas_eraqc",
    "tdps_eraqc",
    "pr_eraqc",
    "sfcWind_eraqc",
    "sfcWind_dir_eraqc",
    "elevation_eraqc",
    "ps_altimeter_eraqc",
    "pr_15min_eraqc",
]
U16_variables = [
    #"raw_qc",
    "qaqc_process",
    "ps_qc",
    "ps_altimeter_qc",
    "psl_qc",
    "tas_qc",
    "tdps_qc",
    "pr_qc",
    "pr_depth_qc",
    "sfcWind_qc",
    "sfcWind_method",
    "sfcWind_dir_qc",
]


In [ ]:
def convert_datatypes(ds):
    """
    Converts the datatypes of variables in a dataset based on external libraries. 
    Used in the station concatenation function.

    Parameters
    ------
        ds: xr.Dataset
            weather station network

    Returns
    -------
        if success:
            output dataset with coverted datatypes
        if failure:
            None
    Notes
    -------
    Uses the following externally defined dictionaries to assign datatypes to variables:
    float32_variables: List
            list of variables that will be converted to datatpe "float64"
    U16_variables: List
            list of variables that will be converted to datatpe "<U16"
    """
    # Generate lists of variables from the external dicionaries that are actually present in the input dataset
    existing_float64 = [
        key for key in float64_variables if key in list(ds.keys())
    ]
    existing_U16 = [key for key in U16_variables if key in list(ds.keys())]

    # Convert the datatypes of those variables, but only if those variables exist
    if len(existing_float64) == 0:
        pass
    else:
        ds[existing_float64] = ds[existing_float64].astype("float64")
    
    if len(existing_U16) == 0:
        pass
    else: 
        ds[existing_U16] = ds[existing_U16].astype("<U16")

    # And of the coordinates as well
    ds.coords["station"] = ds.coords["station"].astype("<U16")

    return ds

In [ ]:
def concatenate_station_pairs(network_name):
    """
    Concatenates two input datasets, deletes the originals, and exports the final concatenated dataset. 
    Also returns a list of the ERA-IDs of all stations that are concatenated.

    Rules
    ------
        1.) concatenation: keep the newer station data in the time range in which both stations overlap

    Parameters
    ------
        network_name: string
            weather station network

    Returns
    -------
        if success: 
            return list of ERA-IDs are stations that are concatenated
            all processed datasets are exported to the merge folder in AWS and the original datasets are deleted
        if failure:
            None
    """
    ##### Read in concatenation list of input network
    network_list = s3_cl.get_object(
        Bucket=bucket,
        Key="3_qaqc_wx/{}_copy/{}/{}_concat_list_TEST.csv".format(
            network_name, network_name, network_name
        ),
    )
    concat_list = pd.read_csv(BytesIO(network_list["Body"].read()))

    subset_number = len(concat_list['concat_subset'].unique())

    # initiate empty list, to which we will iteratively add the ERA-IDs of stations that are concatenated
    final_concat_list = []

    for i in range(0,subset_number):

        # count the number of staions in subset i
        subset_i = concat_list[
            concat_list["concat_subset"].str.contains("{}".format(i))
        ]

        n = subset_i.count()[0]

        # if there are only two stations, proceed with concatenation
        if n == 2:
            # retrieve ERA IDs in this subset of stations
            station_1 = subset_i["ERA-ID"].iloc[0]
            station_2 = subset_i["ERA-ID"].iloc[1]

            final_concat_list.append(station_1)
            final_concat_list.append(station_2)

            # import this subset of datasets and convert to dataframe
            url_1 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
                network_name, station_1
            )
            url_2 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
                network_name, station_2
            )

            # TODO: open_zarr will be used for QAQC'd datasets
            ds_1 = xr.open_zarr(url_1)
            ds_2 = xr.open_zarr(url_2)

            df_1 = ds_1.to_dataframe()
            df_2 = ds_2.to_dataframe()

            # apply reset index only to 'time', as we will need that for concatenation
            df_1 = df_1.reset_index(level="time")
            df_2 = df_2.reset_index(level="time")

            # determine which dataset is older
            if df_1["time"].max() < df_2["time"].max(): 
                # if df_1 has an earlier end tiem than df_2, then d_2 is newer
                # we also grab the name of the newer station in this step, for use later
                df_new = df_2
                ds_new = ds_2

                df_old = df_1
                ds_old = ds_1
            else:
                df_new = df_1
                ds_new = ds_1

                df_old = df_2
                ds_old = ds_2

            # now set things up to determine if there is temporal overlap between df_new and df_old
            df_overlap = df_new[df_new["time"].isin(df_old["time"])]

            # if there is no overlap between the two time series, just concatenate
            if len(df_overlap) == 0:
                df_concat = concat([df_old, df_new])

            # if not, split into subsets and concatenate
            else: 
                ##### Split datframes into subsets #####

                # Remove data in time overlap between old and new
                df_old_cleaned = df_old[~df_old["time"].isin(df_overlap["time"])]
                df_new_cleaned = df_new[~df_new["time"].isin(df_overlap["time"])]

                ##### Concatenate subsets #####
                df_concat = concat([df_old_cleaned, df_overlap, df_new_cleaned])

            ##### Now prepare the final concatenated dataframe for export
            station_name_new = ds_new.coords["station"].values[0]
            final_station_name = "{}".format(station_name_new)
            new_index = [final_station_name] * len(df_concat)
            df_concat.index = new_index
            df_concat.index.name = "station"

            # drop duplicate rows that were potentially generated in the concatenation process
            df_concat = df_concat.drop_duplicates(subset=["time"])

            # Add 'time' back into multi index
            df_concat.set_index("time", append=True, inplace=True)

            # Convert concatenated dataframe to dataset
            ds_concat = df_concat.to_xarray()

            #### Update attributes and datatypes #####

            # Include past attributes
            ds_concat.attrs = ds_new.attrs

            # Update 'history' attribute
            timestamp = datetime.datetime.utcnow().strftime("%m-%d-%Y, %H:%M:%S")
            ds_concat.attrs["history"] = ds_new.attrs[
                "history"
            ] + " \n maritime_merge.ipynb run on {} UTC".format(timestamp)

            # Update 'comment' attribute
            ds_concat.attrs["comment"] = (
                "Final v1 data product. This data has been subjected to cleaning, QA/QC, and standardization."
            )

            # Add new qaqc_files_merged attribute
            station_name_old = ds_old.coords["station"].values[0]
            ds_concat.attrs["qaqc_files_merged"] = (
                "{}_{}, {}_{} merged. Overlap retained from newer station data.".format(
                    network_name, station_name_old, network_name, station_name_new
                )
            )

            # Convert all datatypes, to enable export
            ds_concat = convert_datatypes(ds_concat)

            # ## Export ###
            # export_url = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}_{}.zarr".format(
            #     network_name, "test_concat", network_name # station_name_new
            # )
            # ds_concat.to_zarr(export_url, mode="w")

        # if there are more than two stations in the subset, continue
        else:
            continue

    return final_concat_list  
    # return df_concat, df_new, df_old, df_overlap

### CHECK

In [ ]:
network_name = "VALLEYWATER"
final_concat_list = concatenate_station_pairs(network_name)

Import

In [ ]:
network_list = s3_cl.get_object(
    Bucket=bucket,
    Key="3_qaqc_wx/{}_copy/{}/{}_concat_list_TEST.csv".format(
        network_name, network_name, network_name
    ),
)
concat_list = pd.read_csv(BytesIO(network_list["Body"].read()))

In [ ]:
station_1 = concat_list["ERA-ID"].iloc[0]
station_2 = concat_list["ERA-ID"].iloc[1]

# import this subset of datasets and convert to dataframe
url_1 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
    network_name, station_1
)
url_2 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
    network_name, station_2
)

ds_1 = xr.open_zarr(url_1)
ds_2 = xr.open_zarr(url_2)

df_1_original = ds_1.to_dataframe()
df_2_original = ds_2.to_dataframe()

In [ ]:
# import output
url_output = "s3://wecc-historical-wx/3_qaqc_wx/{}/test_concat_{}.zarr".format(
    network_name, network_name
)

# TODO: open_zarr will be used for QAQC'd datasets
ds_concat = xr.open_zarr(url_output)

df_concat_original = ds_concat.to_dataframe()

Does the concatenated dataframe contain the input dataframes?

In [ ]:
# Check if input_df is a subset of concatenated_df
is_subset = df_1_original.isin(df_concat_original).all().all()

print("Input DataFrame is a subset of concatenated DataFrame:", is_subset)

In [ ]:
# extract time index for plotting
df_1 = df_1_original.reset_index(level="time")
df_2 = df_2_original.reset_index(level="time")
df_concat = df_concat_original.reset_index(level="time")

In [ ]:
# Check if time range of new dataframe is in the concatenated dataframe
is_subset = df_1['time'].isin(df_concat['time']).all().all()

print("Input DataFrame is a subset of concatenated DataFrame:", is_subset)

In [ ]:

# now set things up to determine if there is temporal overlap between df_new and df_old
df_1_overlap = df_1[df_1["time"].isin(df_concat["time"])]

In [ ]:
df_concat_overlap = df_concat[df_concat["time"].isin(df_1["time"])]

In [ ]:
df_1_overlap["lat"] = df_1_overlap["lat"].round(3)
df_1_overlap["lon"] = df_1_overlap["lon"].round(3)

In [ ]:
df_concat_overlap["lat"] = df_concat_overlap["lat"].round(3)
df_concat_overlap["lon"] = df_concat_overlap["lon"].round(3)

In [ ]:
df_concat_overlap.head()

In [ ]:
df_1_overlap.head()

Plot the two original datasets

In [ ]:
# Create a figure with a specific size
plt.figure(figsize=(8, 4))

# Plotting the time series of given dataframe
plt.plot(df_1["time"], df_1["pr_15min"])

# Plotting the time series of given dataframe
plt.plot(df_2["time"], df_2["pr_15min"])

# Giving title to the chart using plt.title
plt.title("input dfs")

# rotating the x-axis tick labels at 30degree
# towards right
plt.xticks(rotation=30, ha="right")

# Providing x and y label to the chart
plt.xlabel("time")
plt.ylabel("pr_15min")

plt.show()

Plot the output dataset

In [ ]:
# Create a figure with a specific size
plt.figure(figsize=(8, 4))

# Plotting the time series of given dataframe
plt.plot(df_concat["time"], df_concat["pr_15min"])

# Giving title to the chart using plt.title
plt.title("concatenated df")

# rotating the x-axis tick labels at 30degree
# towards right
plt.xticks(rotation=30, ha="right")

# Providing x and y label to the chart
plt.xlabel("time")
plt.ylabel("pr_15min")

plt.show()

### Previous Approaches

In [ ]:
def concat_test(concat_list):
    """
    Performs concatenation for stations in list of stations flagged for concatenation.

    Rules
    ------
        1.) concatenation: keep the newer station data in the time range in which both stations overlap
    Parameters
    ------
        network_name: string
            weather station network
        station_old: string
            name of the older weather station
        station_new: string
            name of the newer weather station
    Returns
    -------
        if success:
            all processed datasets are exported to the merge folder in AWS and the original datasets are deleted
        if failure:
            None
    """
    ##### Import target datasets and convert to dataframe
    flag_range = list(
        range(concat_list["concat_flag"].min(), concat_list["concat_flag"].max())
    )

    for i in flag_range:
        subset_list = concat_list[concat_list["concat_flag"] == i]
        subset_range = list(range(0, len(subset_list)))

        url = {}
        ds = {}
        df = {}

        for i in subset_range:

            # extract information needed for dataset import
            row_i = subset_list.iloc[[i]]
            network_name = row_i["ERA-ID"].split("_")[
                0
            ]  # TODO: this does not work, when it really should
            station_name = row_i["ERA-ID"]

            url[i] = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}_{}.zarr".format(
                network_name, network_name, station_name
            )

            ds[i] = xr.open_zarr(url[i])

            df[i] = ds[i].to_dataframe()

            # Apply reset index only to 'time', as we will need that for concatenation
            df[i] = df[i].reset_index(level="time")

    ##### Split datframes into subsets #####

    # Remove data in time overlap between old and new
    df_old_cleaned = df_old[~df_old["time"].isin(df_new["time"])]
    df_new_cleaned = df_new[~df_new["time"].isin(df_old["time"])]

    # Data in new input that overlaps in time with old input
    df_overlap = df_new[df_new["time"].isin(df_old["time"])]

    # Set index to new input for df_old_cleaned
    # We want the final dataset to show up as the new station, not the old
    final_station_name = "{}_{}".format(network_name, station_new)
    new_index = [final_station_name] * len(df_old_cleaned)

    df_old_cleaned.index = new_index
    df_old_cleaned.index.name = "station"

    ##### Concatenate subsets #####

    df_concat = concat([df_old_cleaned, df_overlap, df_new_cleaned])

    # Add 'time' back into multi index
    df_concat.set_index("time", append=True, inplace=True)

    # Convert concatenated dataframe to dataset
    ds_concat = df_concat.to_xarray()

    ##### Update attributes and datatypes #####

    # Include past attributes
    ds_concat.attrs = ds_new.attrs

    # Update 'history' attribute
    timestamp = datetime.datetime.utcnow().strftime("%m-%d-%Y, %H:%M:%S")
    ds_concat.attrs["history"] = ds_new.attrs[
        "history"
    ] + " \nmaritime_merge.ipynb run on {} UTC".format(timestamp)

    # Update 'comment' attribute
    ds_concat.attrs["comment"] = (
        "Final v1 data product. This data has been subjected to cleaning, QA/QC, and standardization."
    )

    # Add new qaqc_files_merged attribute
    ds_concat.attrs["qaqc_files_merged"] = (
        "{}_{}, {}_{} merged. Overlap retained from newer station data.".format(
            network_name, station_old, network_name, station_new
        )
    )

    # Convert all datatypes, to enable export
    existing_float32 = [col for col in float32_variables if col in df_concat.columns]
    existing_U16 = [col for col in U16_variables if col in df_concat.columns]

    ds_concat[existing_float32] = ds_concat[existing_float32].astype("float32")
    ds_concat[existing_U16] = ds_concat[existing_U16].astype("U16")

    ds_concat.coords["station"] = ds_concat.coords["station"].astype("<U16")

    ### Export ###

    # delete old inputs
    bucket = "wecc-historical-wx"
    key_new = "4_merge_wx/{}_dev/{}_{}.zarr".format(
        network_name, network_name, station_new
    )
    key_old = "4_merge_wx/{}_dev/{}_{}.zarr".format(
        network_name, network_name, station_old
    )

    delete_folder(bucket, key_new)
    delete_folder(bucket, key_old)

    # Export final, concatenated dataset
    export_url = "s3://wecc-historical-wx/4_merge_wx/{}_dev/{}_{}.zarr".format(
        network_name, network_name, "test"
    )
    ds_concat.to_zarr(export_url, mode="w")

    return None  # ds_concat

## CODE SCRAPS

In [ ]:
test = asosawos_list_concat.groupby(["ICAO"]).apply(
    lambda x: x.sort_values(["end_time"])
)

In [ ]:
# # sort by end_time or end-date, depending on the station TODO: this is not necessary
# time_var_list = ['end_time','end-date']
# end_time_or_date = [col for col in station_list.columns if col in time_var_list]
# new_station_list = new_station_list.groupby('concat_flag').apply(lambda x: x.sort_values(end_time_or_date)).reset_index(drop=True)

In [ ]:
def concatenate_target_stations_old(df):
    """
    Concatenates station data that has been flagged for concatenation

    Rules
    ------
        1.)

    Parameters
    ------
        df: pd.dataframe
            staton data

    Returns
    -------
        if success:
            all processed datasets are exported to the merge folder in AWS and the original datasets are deleted
        if failure:
            None
    """

    # Apply reset index only to 'time', as we will need that for concatenation
    df_old = df_old.reset_index(level="time")
    df_new = df_new.reset_index(level="time")

    ##### Split datframes into subsets #####
    # if there is overlap, then create subsets

    # if no overlap, just concatenate

    # Remove data in time overlap between old and new
    df_old_cleaned = df_old[~df_old["time"].isin(df_new["time"])]
    df_new_cleaned = df_new[~df_new["time"].isin(df_old["time"])]

    # Data in new input that overlaps in time with old input
    df_overlap = df_new[df_new["time"].isin(df_old["time"])]

    # Set index to new input for df_old_cleaned
    # We want the final dataset to show up as the new station, not the old
    final_station_name = "{}_{}".format(network_name, station_new)
    new_index = [final_station_name] * len(df_old_cleaned)

    df_old_cleaned.index = new_index
    df_old_cleaned.index.name = "station"

    ##### Concatenate subsets #####

    df_concat = concat([df_old_cleaned, df_overlap, df_new_cleaned])

    # Add 'time' back into multi index
    df_concat.set_index("time", append=True, inplace=True)

    # Convert concatenated dataframe to dataset
    ds_concat = df_concat.to_xarray()

    ##### Update attributes and datatypes #####

    # Include past attributes
    ds_concat.attrs = ds_new.attrs

    # Update 'history' attribute
    timestamp = datetime.datetime.utcnow().strftime("%m-%d-%Y, %H:%M:%S")
    ds_concat.attrs["history"] = ds_new.attrs[
        "history"
    ] + " \nmaritime_merge.ipynb run on {} UTC".format(timestamp)

    # Update 'comment' attribute
    ds_concat.attrs["comment"] = (
        "Final v1 data product. This data has been subjected to cleaning, QA/QC, and standardization."
    )

    # Add new qaqc_files_merged attribute
    ds_concat.attrs["qaqc_files_merged"] = (
        "{}_{}, {}_{} merged. Overlap retained from newer station data.".format(
            network_name, station_old, network_name, station_new
        )
    )

    # Convert all datatypes, to enable export
    existing_float32 = [col for col in float32_variables if col in df_concat.columns]
    existing_U16 = [col for col in U16_variables if col in df_concat.columns]

    ds_concat[existing_float32] = ds_concat[existing_float32].astype("float32")
    ds_concat[existing_U16] = ds_concat[existing_U16].astype("U16")

    ds_concat.coords["station"] = ds_concat.coords["station"].astype("<U16")

    return None  # ds_concat

In [ ]:
def reorder_station_list(station_list, concat_list, duplicate_list):
    """
    Reorders the input station list, necessary for concatenation

    Rules
    ------
        1.)

    Parameters
    ------
        station_list: pd.dataframe

        concat_list: pd.dataframe

        duplicate_list: pd.dataframe

    Returns
    -------
        if success:
            output station list with stations to be concatenated at top, followed by potential duplicates
        if failure:
            None
    """

    ##### subsets of station list

    # stations that will be concatenated
    concat_stations = station_list[station_list["ICAO"].isin(concat_list)]

    # potential duplicate stations
    duplicate_stations = station_list[station_list["ICAO"].isin(duplicate_list)]

    # all remaining stations
    remaining_stations = station_list[
        ~station_list["ICAO"].isin(duplicate_list + concat_list)
    ]

    ##### sort concat list alphabetically, to ensure that stations with the same ICAO are grouped together
    concat_stations = concat_stations.sort_values("ICAO")
    duplicate_stations = duplicate_stations.sort_values("ICAO")

    ##### now within each ICAO, order by end time
    concat_stations = concat_stations.groupby(["ICAO"]).apply(
        lambda x: x.sort_values(["end_time"])
    )

    ##### concatenate susbets and reset index
    new_list = concat(
        [concat_stations, duplicate_stations, remaining_stations]
    ).reset_index(drop=True)

    return new_list

In [ ]:
# check for presence of start and end times

time_check = repeat_list_subset.groupby("ICAO").apply(lambda x: x.isnull().any())

print("number of null start times:")
print(time_check["start_time"].sum())

print("number of null end times:")
print(time_check["end_time"].sum())

In [ ]:
# check if the start and end times are identical

start_duplicate_check = (
    repeat_list_subset.groupby("ICAO")
    .apply(lambda x: x.duplicated(subset=["start_time"]))
    .rename("check")
    .reset_index()
)
end_duplicate_check = (
    repeat_list_subset.groupby("ICAO")
    .apply(lambda x: x.duplicated(subset=["end_time"]))
    .rename("check")
    .reset_index()
)

In [ ]:
end_list = end_duplicate_check[end_duplicate_check["check"] == True]["ICAO"].tolist()
start_list = start_duplicate_check[start_duplicate_check["check"] == True][
    "ICAO"
].tolist()

print(end_list)
print(start_list)

In [ ]:
# what is going on with the stations that have duplicate start and end times? are they true duplicates?

repeat_list_subset[repeat_list_subset["ICAO"].isin(start_list + end_list)]

In [ ]:
# load in single dc file from AWS
ds_1 = read_nc_from_s3_clean("ASOSAWOS", "ASOSAWOS_72026294076", temp_dir)
ds_2 = read_nc_from_s3_clean("ASOSAWOS", "ASOSAWOS_A0000594076", temp_dir)


# convert to formatted pandas dataframe
df_1 = qaqc_ds_to_df(ds_1, verbose=False)
df_2 = qaqc_ds_to_df(ds_2, verbose=False)

In [ ]:
lon = df_1.lon.mean()
lat = df_1.lat.mean()
# print("{}, {:.5f}, {:.5f}".format(id, lon, lat))


# Plot time series of the data
fig, ax = plt.subplots(figsize=(9, 3))

df_1.plot(ax=ax, x="time", y="sfcWind")
df_2.plot(ax=ax, x="time", y="sfcWind")

ax.set_title("{}  ({:.3f}, {:.3f})".format(id, lon, lat))

In [ ]:
def matching_check_old(station_list):
    """
    Resamples meteorological variables to hourly timestep according to standard conventions.

    Rules
    ------
        1.)

    Parameters
    ------
        df: pd.DataFrame
            list of station information

    Returns
    -------
        if success:
            list
                list of ICAO values of stations that need to be concatenated
            list
                list of ICAO values of potential duplicate stations
        if failure:
            None
    """
    # Generate list of repeat ICAOs
    repeat_list = station_list[station_list.duplicated(subset=["ICAO"], keep=False)]
    repeat_list = repeat_list[
        ["ICAO", "ERA-ID", "STATION NAME", "start_time", "end_time"]
    ]

    concat_list = repeat_list["ICAO"].unique().tolist()

    # And empty list to add potential duplicates to
    duplicate_list = []

    ##### Generate boolean for whether or not there are null start and/or end times
    # TODO: may not be necessary
    time_check = repeat_list.groupby("ICAO").apply(lambda x: x.isnull().any())

    end_nan_list = time_check[time_check["end_time"] == True]["ICAO"].tolist()
    start_nan_list = time_check[time_check["start_time"] == True]["ICAO"].tolist()

    # add ICAOs of stations with nan start or end times to potential duplicates list
    duplicate_list = duplicate_list + start_nan_list + end_nan_list

    duplicate_list = duplicate_list

    ##### Identify ICAOs with duplicate start end times
    start_duplicate_check = (
        repeat_list.groupby("ICAO")
        .apply(lambda x: x.duplicated(subset=["start_time"]))
        .rename("check")
        .reset_index()
    )
    end_duplicate_check = (
        repeat_list.groupby("ICAO")
        .apply(lambda x: x.duplicated(subset=["end_time"]))
        .rename("check")
        .reset_index()
    )

    end_dup_list = end_duplicate_check[end_duplicate_check["check"] == True][
        "ICAO"
    ].tolist()
    start_dup_list = start_duplicate_check[start_duplicate_check["check"] == True][
        "ICAO"
    ].tolist()

    # add ICAOs of stations with nan start or end times to potential duplicates list
    duplicate_list = duplicate_list + start_dup_list + end_dup_list

    # Generate final list of ICAOs for stations to be concatenated
    concat_list = [x for x in concat_list if x not in duplicate_list]

    return concat_list, duplicate_list

In [ ]:
# order the subset with only stations to concatenate

asosawos_list_concat["ICAO"] = pd.Categorical(
    asosawos_list_concat["ICAO"], categories=concat_list, ordered=True
)

test_list = asosawos_list_concat.sort_values("ICAO").reset_index(drop=True)

### Stations within a certain distance

In [ ]:
# Convert the data into GeoDataFrames
# using EPSG 3310

gdf_asosawos = gpd.GeoDataFrame(
    asosawos_list,
    geometry=[
        Point(lon, lat) for lon, lat in zip(asosawos_list["LON"], asosawos_list["LAT"])
    ],
    crs="EPSG:4326",
).to_crs(epsg=3310)

#### approach 3: find the nearest point in the geodataframe

In [ ]:
# insert emtpy columns

gdf_asosawos["nearest_station"] = pd.Series(dtype="U16")
gdf_asosawos["distance"] = pd.Series(dtype="float32")

In [ ]:
for index, row in gdf_asosawos.iterrows():
    # geometry of individual row 
    point = row.geometry
    # returns a multipoint object with the geometries of every row in the gdf
    multipoint = gdf_asosawos.drop(index, axis=0).geometry.unary_union
    # 
    queried_geom, nearest_geom = nearest_points(point, multipoint)
    dist_from_point = 
    gdf_asosawos.loc[index, 'nearest_geometry'] = nearest_geom
    gdf_asosawos.loc[index, 'distance'] = nearest_geom

#### approach 2: distance function

In [ ]:
## function to calculate the distance between points


def distance_sort_filter(row, df2, buffer=None, id=False):

    dist = df2.geometry.distance(row).sort_values()

    if buffer:
        dist = dist[dist < buffer]

    if id:
        distances = {
            df2.loc[idx]["WBAN"]: value for idx, value in zip(dist.index, dist.values)
        }
    else:
        distances = {idx: value for idx, value in zip(dist.index, dist.values)}

    return distances

#### approach 1: using sjoin

In [ ]:
# Create a buffer around points in gdf1 (e.g., 10 km buffer)
gdf_asosawos["buffer"] = gdf_asosawos.geometry.buffer(
    0.1
)  # Buffer in degrees, 0.1 degrees approx equals 10 km

In [ ]:
# Perform a spatial join using the buffer
merged = gpd.sjoin(
    gdf_asosawos, gdf_asosawos[["geometry", "buffer"]], how="inner", predicate="within"
)

# The 'merged' GeoDataFrame contains points from gdf_isd that are within the buffer around points in gdf_asosawos

In [ ]:
print(
    merged
)  # there are not ISD stations within 10km of an ASOSAWOS station missed by the exact matching

In [ ]:
### Round asosawos down to 3 decimal points of accuracy
# asosawos_round = asosawos_list.round({"LAT": 3, "LON": 3})

Potential ways to check that two stations are duplicates
1. identical total_nobs
2. identical ERA IDs
3. identical end or start times 

In [ ]:
# Extract flagged stations

asosawos_dup = asosawos_out[~asosawos_out["concat_flag"].isna()]
valleywater_dup = valleywater_out[~valleywater_out["concat_flag"].isna()]
maritime_dup = maritime_out[~maritime_out["concat_flag"].isna()]

### Final Function

In [ ]:
def duplicate_check(station_list):
    """
    This function flags stations that are potentially duplicates

    Rules
    ------
        1.) Within stations flagged for concatenation, stations are flagged as potential duplicates
            if either their start or end times are identical
            - TODO: brainstorm alternative approaches

    Parameters
    ------
        station_list: pd.DataFrame
            list of station information that has passed through the concatenation check

    Returns
    -------
        if success:
            new_station_list: pd.DataFrame


        if failure:
            None
    Notes
    -------

    """
    ##### flag stations with repeat end or start times

    time_end_list = ["end_time", "end-date"]
    time_start_list = ["start_time", "start-date"]

    end_time_or_date = [col for col in station_list.columns if col in time_var_list]

    new_station_list = (
        new_station_list.groupby("concat_flag")
        .apply(lambda x: x.sort_values(end_time_or_date))
        .reset_index(drop=True)
    )

    return new_station_list

In [ ]:
def concatenate_target_stations_old(network_name, station_old, station_new):
    """
    Concatenates two input datasets, deletes the originals, and exports the final concatenated dataset

    Rules
    ------
        1.) concatenation: keep the newer station data in the time range in which both stations overlap

    Parameters
    ------
        network_name: string
            weather station network
        station_old: string
            name of the older weather station
        station_new: string
            name of the newer weather station

    Returns
    -------
        if success:
            all processed datasets are exported to the merge folder in AWS and the original datasets are deleted
        if failure:
            None
    """
    # Import target datasets and convert to dataframe
    old_url = "s3://wecc-historical-wx/4_merge_wx/{}_dev/{}_{}.zarr".format(
        network_name, network_name, station_old
    )
    new_url = "s3://wecc-historical-wx/4_merge_wx/{}_dev/{}_{}.zarr".format(
        network_name, network_name, station_new
    )

    ds_old = xr.open_zarr(old_url)
    ds_new = xr.open_zarr(new_url)

    df_old = ds_old.to_dataframe()
    df_new = ds_new.to_dataframe()

    # Apply reset index only to 'time', as we will need that for concatenation
    df_old = df_old.reset_index(level="time")
    df_new = df_new.reset_index(level="time")

    ##### Split datframes into subsets #####

    # Remove data in time overlap between old and new
    df_old_cleaned = df_old[~df_old["time"].isin(df_new["time"])]
    df_new_cleaned = df_new[~df_new["time"].isin(df_old["time"])]

    # Data in new input that overlaps in time with old input
    df_overlap = df_new[df_new["time"].isin(df_old["time"])]

    # Set index to new input for df_old_cleaned
    # We want the final dataset to show up as the new station, not the old
    final_station_name = "{}_{}".format(network_name, station_new)
    new_index = [final_station_name] * len(df_old_cleaned)

    df_old_cleaned.index = new_index
    df_old_cleaned.index.name = "station"

    ##### Concatenate subsets #####

    df_concat = concat([df_old_cleaned, df_overlap, df_new_cleaned])

    # Add 'time' back into multi index
    df_concat.set_index("time", append=True, inplace=True)

    # Convert concatenated dataframe to dataset
    ds_concat = df_concat.to_xarray()

    ##### Update attributes and datatypes #####

    # Include past attributes
    ds_concat.attrs = ds_new.attrs

    # Update 'history' attribute
    timestamp = datetime.datetime.utcnow().strftime("%m-%d-%Y, %H:%M:%S")
    ds_concat.attrs["history"] = ds_new.attrs[
        "history"
    ] + " \nmaritime_merge.ipynb run on {} UTC".format(timestamp)

    # Update 'comment' attribute
    ds_concat.attrs["comment"] = (
        "Final v1 data product. This data has been subjected to cleaning, QA/QC, and standardization."
    )

    # Add new qaqc_files_merged attribute
    ds_concat.attrs["qaqc_files_merged"] = (
        "{}_{}, {}_{} merged. Overlap retained from newer station data.".format(
            network_name, station_old, network_name, station_new
        )
    )

    # Convert all datatypes, to enable export
    existing_float32 = [col for col in float32_variables if col in df_concat.columns]
    existing_U16 = [col for col in U16_variables if col in df_concat.columns]

    ds_concat[existing_float32] = ds_concat[existing_float32].astype("float32")
    ds_concat[existing_U16] = ds_concat[existing_U16].astype("U16")

    ds_concat.coords["station"] = ds_concat.coords["station"].astype("<U16")

    ### Export ###

    # delete old inputs
    bucket = "wecc-historical-wx"
    key_new = "4_merge_wx/{}_dev/{}_{}.zarr".format(
        network_name, network_name, station_new
    )
    key_old = "4_merge_wx/{}_dev/{}_{}.zarr".format(
        network_name, network_name, station_old
    )

    delete_folder(bucket, key_new)
    delete_folder(bucket, key_old)

    # Export final, concatenated dataset
    export_url = "s3://wecc-historical-wx/4_merge_wx/{}_dev/{}_{}.zarr".format(
        network_name, network_name, "test"
    )
    ds_concat.to_zarr(export_url, mode="w")

    return None  # ds_concat

#### original function

In [ ]:
def concatenate_station_pairs(network_name):
    """
    Concatenates two input datasets, deletes the originals, and exports the final concatenated dataset. 
    Also returns a list of the ERA-IDs of all stations that are concatenated.

    Rules
    ------
        1.) concatenation: keep the newer station data in the time range in which both stations overlap

    Parameters
    ------
        network_name: string
            weather station network

    Returns
    -------
        if success: 
            return list of ERA-IDs are stations that are concatenated
            all processed datasets are exported to the merge folder in AWS and the original datasets are deleted
        if failure:
            None
    """
    ##### Read in concatenation list of input network
    network_list = s3_cl.get_object(
        Bucket=bucket,
        Key="3_qaqc_wx/{}/concat_list_{}.csv".format(
            network_name, network_name, network_name
        ),
    )
    concat_list = pd.read_csv(BytesIO(network_list["Body"].read()))

    # ! you can truncate the concat list here, for testing
    concat_list = concat_list.head(2)
    # ! end

    subset_number = len(concat_list['concat_subset'].unique())

    # initiate empty list, to which we will iteratively add the ERA-IDs of stations that are concatenated
    final_concat_list = []

    for i in range(0,subset_number):

        # count the number of staions in subset i
        subset_i = concat_list[
            concat_list["concat_subset"].str.contains("{}".format(i))
        ]

        n = subset_i.count()[0]

        # if there are only two stations, proceed with concatenation
        if n == 2:
            try: 
                # retrieve ERA IDs in this subset of stations
                station_1 = subset_i["ERA-ID"].iloc[0]
                station_2 = subset_i["ERA-ID"].iloc[1]

                # import this subset of datasets and convert to dataframe
                url_1 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
                    network_name, station_1
                )
                url_2 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(
                    network_name, station_2
                )

                ds_1 = xr.open_zarr(url_1)
                ds_2 = xr.open_zarr(url_2)

                df_1,MultiIndex_1,attrs_1,var_attrs_1,era_qc_vars_1 = qaqc_ds_to_df(ds_1, verbose=False)
                df_2, MultiIndex_2, attrs_2, var_attrs_2, era_qc_vars_2 = qaqc_ds_to_df(ds_2, verbose=False)

                # determine which dataset is older
                if df_1["time"].max() < df_2["time"].max():
                    # if df_1 has an earlier end tiem than df_2, then d_2 is newer
                    # we also grab the name of the newer station in this step, for use later
                    df_new = df_2
                    ds_new = ds_2
                    MultiIndex_new = MultiIndex_2
                    attrs_new = attrs_2

                    df_old = df_1
                    ds_old = ds_1
                    MultiIndex_old = MultiIndex_1

                else:
                    df_new = df_1
                    ds_new = df_1
                    MultiIndex_new = MultiIndex_2
                    attrs_new = attrs_2

                    df_old = df_2
                    ds_old = ds_2
                    MultiIndex_old = MultiIndex_2

                # now set things up to determine if there is temporal overlap between df_new and df_old
                df_overlap = df_new[df_new["time"].isin(df_old["time"])]

                # if there is no overlap between the two time series, just concatenate
                if len(df_overlap) == 0:
                    df_concat = concat([df_old, df_new])

                # if not, split into subsets and concatenate
                else:
                    ##### Split datframes into subsets #####

                    # Remove data in time overlap between old and new
                    df_old_cleaned = df_old[~df_old["time"].isin(df_overlap["time"])]
                    df_new_cleaned = df_new[~df_new["time"].isin(df_overlap["time"])]

                    ##### Concatenate subsets #####
                    df_concat = concat([df_old_cleaned, df_overlap, df_new_cleaned])

                # ##### Now prepare the final concatenated dataframe for export
                station_name_new = MultiIndex_new.get_level_values("station")[1]
                
                # ! This is where Neil and I made the change to address the issues
                # ! 
                MultiIndex_old = pd.MultiIndex.from_tuples(
                    [(station_name_new, lvl1) for _, lvl1 in MultiIndex_old],
                    names=MultiIndex_new.names,
                )

                MultiIndex_concat = MultiIndex_new.union(MultiIndex_old)

                # drop duplicate rows that were potentially generated in the concatenation process
                df_concat = df_concat.drop_duplicates(subset=["time"])

                # drop 'station' and 'time'columns
                df_concat = df_concat.drop(["station", "time","hour","day","month","year","date"], axis=1)

                print('length of MultiIndex_new')
                print(len(MultiIndex_new))
                print("length of MultiIndex_old")
                print(len(MultiIndex_old))
                print("length of MultiIndex_concat")
                print(len(MultiIndex_concat))

                print("length of df_new")
                print(len(df_new))
                print("length of df_old")
                print(len(df_old))
                print("length of df_concat")
                print(len(df_concat))

                # ! This is where the issue! MultiIndex_concat and df_concat have difference lengths
                df_concat.index = MultiIndex_concat

                # # Convert concatenated dataframe to dataset
                # ds_concat = df_concat.to_xarray()

                # # #### Prepare for export #####

                # # Convert datatype of station coordinate
                # ds_concat.coords["station"] = ds_concat.coords["station"].astype("<U20")

                # # # Include past attributes
                # ds_concat.attrs.update(attrs_new)

                # # Update 'history' attribute
                # timestamp = datetime.datetime.utcnow().strftime("%m-%d-%Y, %H:%M:%S")
                # ds_concat.attrs["history"] = ds_concat.attrs[
                #     "history"
                # ] + " \n maritime_merge.ipynb run on {} UTC".format(timestamp)

                # # Update 'comment' attribute
                # ds_concat.attrs["comment"] = (
                #     "Final v1 data product. This data has been subjected to cleaning, QA/QC, and standardization."
                # )

                # # Add new qaqc_files_merged attribute
                # station_name_old = MultiIndex_old.get_level_values("station")[1]
                # ds_concat.attrs["qaqc_files_merged"] = (
                #     "{}, {} merged. Overlap retained from newer station data.".format(
                #         station_name_old, station_name_new
                #     )
                # )

                # ! this is here the renaming will go

                # !

                # ## Export ###
                # ! a test name is used below
                # ! the final name will be that of the newer dataframe
                # export_url = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}_{}.zarr".format(
                #     network_name, "test_concat", station_name_new
                # )
                # ds_concat.to_zarr(export_url, mode="w")

                # record that the stations were concatenated
                final_concat_list.append(station_1)
                final_concat_list.append(station_2)

            except Exception as e:
                print(
                    "Error concatenating subset {}: {}".format(subset_i, e)
                )
        # if there are more than two stations in the subset, continue
        else:
            continue

    # return final_concat_list # ! this will be the final return statement, below is inlcluded for testing
    # return (
    #     df_new,
    #     df_old,
    #     df_concat,
    #     ds_concat,
    #     final_concat_list,
    # )

    return df_1, df_2, MultiIndex_1, MultiIndex_2, df_concat

### TEST

In [ ]:
network_name = "MARITIME" # "VALLEYWATER", "MARITIME"

In [ ]:
df_1, df_2, MultiIndex_1, MultiIndex_2 = concatenate_station_pairs(network_name)

In [ ]:
# LJAC1 - this should be new
print(df_1['time'].min())
print(df_1["time"].max())

In [ ]:
# LJPC1
print(df_2["time"].min())
print(df_2["time"].max())

In [ ]:
# determine which dataset is older
if df_2["time"].max() > df_1["time"].max():
    # if df_1 has an earlier end tiem than df_2, then d_2 is newer
    # we also grab the name of the newer station in this step, for use later
    df_new = df_2
    MultiIndex_new = MultiIndex_2

    df_old = df_1
    MultiIndex_old = MultiIndex_1

else:
    df_new = df_1
    ds_new = df_1
    MultiIndex_new = MultiIndex_1

    df_old = df_2
    MultiIndex_old = MultiIndex_2

# now set things up to determine if there is temporal overlap between df_new and df_old
df_overlap = df_new[df_new["time"].isin(df_old["time"])]

In [ ]:
# if there is no overlap between the two time series, just concatenate
if len(df_overlap) == 0:
    df_concat = concat([df_old, df_new])

# if not, split into subsets and concatenate
else:
    ##### Split datframes into subsets #####

    # Remove data in time overlap between old and new
    df_old_cleaned = df_old[~df_old["time"].isin(df_overlap["time"])]
    df_new_cleaned = df_new[~df_new["time"].isin(df_overlap["time"])]

    ##### Concatenate subsets #####
    df_concat = concat([df_old_cleaned, df_overlap, df_new_cleaned])

In [ ]:
# ##### Now prepare the final concatenated dataframe for export
station_name_new = MultiIndex_new.get_level_values("station")[1]

MultiIndex_old = pd.MultiIndex.from_tuples(
    [(station_name_new, lvl1) for _, lvl1 in MultiIndex_old],
    names=MultiIndex_new.names,
)

MultiIndex_concat = MultiIndex_new.union(MultiIndex_old)


# MultiIndex_concat = pd.MultiIndex.from_tuples(
#     [(station_name_new, lvl1) for _, lvl1 in MultiIndex_concat],
#     names=MultiIndex_concat.names,
# )

In [ ]:
# drop duplicate rows that were potentially generated in the concatenation process
df_concat = df_concat.drop_duplicates(subset=["time"])

# drop 'station' and 'time'columns
df_concat = df_concat.drop(["station", "time","hour","day","month","year","date"], axis=1)

df_concat.index = MultiIndex_concat

# Convert concatenated dataframe to dataset
ds_concat = df_concat.to_xarray()

In [ ]:
ds_concat

Union is the issue - mismatch in timesteps

In [ ]:
print(df_1['time'].min())
print(df_1["time"].max())

In [ ]:
print(df_2["time"].min())
print(df_2["time"].max())

df_concat should span 2005-01-01 01:30:00 - 2022-08-31 23:54:00

In [ ]:
len(MultiIndex_concat)

In [ ]:
len(df_concat)

In [ ]:
df_concat.columns

In [ ]:
# drop duplicate rows that were potentially generated in the concatenation process
df_concat_drop_dups = df_concat.drop_duplicates(subset=["time"])

In [ ]:
len(df_concat_drop_dups)

In [ ]:
# MultiIndex_concat and df_concat_drop_dups['time']
index_time = list(MultiIndex_1.get_level_values("time"))
#df_time = list(df_concat_drop_dups['time'])

In [ ]:
MultiIndex_1.get_level_values("time")

In [ ]:
df_new['time']

In [ ]:
df_concat['time']

In [ ]:
dups = df_concat[df_concat['time'].duplicated(keep=False)]

In [ ]:
dups

In [ ]:
# drop 'station' and 'time'columns
df_concat = df_concat.drop(
    ["station", "time", "hour", "day", "month", "year", "date"], axis=1
)

df_concat.index = MultiIndex_concat

#### Test option 1

Run concatenate_station_pairs() as is, so the function does not export and instead returns df_concat, df_new, df_old, and df_overlap

In [ ]:
(
    df_new,
    df_old,
    df_concat,
    ds_concat,
    final_concat_list,
) = concatenate_station_pairs(network_name)

In [ ]:
df_concat = df_concat.reset_index(level="time")

#### Test option 2: 

Run concatenate_station_pairs() with the first return statement uncommented and the second commented, and the export section uncommented. So that the function actually exports the concatenated datasets. I've generated all the concatention lists (for VALLEYWATER, MARITIME, and ASOSAWOS) needed to run the function.

In [ ]:
output = concatenate_station_pairs(network_name)

In [ ]:
# import output
# TODO: you'll need to change the url
url_output = "s3://wecc-historical-wx/3_qaqc_wx/{}/test_concat_{}.zarr".format(
    network_name, network_name
)

# TODO: open_zarr will be used for QAQC'd datasets
ds_concat = xr.open_zarr(url_output)

df_concat = ds_concat.to_dataframe()

In [ ]:
network_list = s3_cl.get_object(
    Bucket=bucket,
    Key="3_qaqc_wx/{}/{}_concat_list_{}.csv".format(
        network_name, network_name, network_name
    ),
)
concat_list = pd.read_csv(BytesIO(network_list["Body"].read()))
station_1 = concat_list["ERA-ID"].iloc[0]
station_2 = concat_list["ERA-ID"].iloc[1]

# import this subset of datasets and convert to dataframe
url_1 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(network_name, station_1)
url_2 = "s3://wecc-historical-wx/3_qaqc_wx/{}/{}.zarr".format(network_name, station_2)

ds_1 = xr.open_zarr(url_1)
ds_2 = xr.open_zarr(url_2)

df_1 = ds_1.to_dataframe()
df_2 = ds_2.to_dataframe()

In [ ]:
# extract time index for plotting
df_1 = df_1.reset_index(level="time")
df_2 = df_2.reset_index(level="time")


df_concat = df_concat.reset_index(level="time")

In [ ]:
if df_1["time"].max() < df_2["time"].max(): 
    # if df_1 has an earlier end tiem than df_2, then d_2 is newer
    # we also grab the name of the newer station in this step, for use later
    df_new = df_2
    ds_new = ds_2

    df_old = df_1
    ds_old = ds_1
else:
    df_new = df_1
    ds_new = ds_1

    df_old = df_2
    ds_old = ds_2

#### Onward

In [ ]:
ds_concat

In [ ]:
df_concat.head(4)

Check overlap

In [ ]:
# now set things up to determine if there is temporal overlap between df_new and df_old
df_new_overlap = df_new[df_new["time"].isin(df_concat["time"])]
df_concat_overlap = df_concat[df_concat["time"].isin(df_new["time"])]

In [ ]:
df_new_overlap.head(4)

In [ ]:
df_concat_overlap.head(4)

Plot the two original datasets

In [ ]:
vis_var = 'ps'

In [ ]:
# Create a figure with a specific size
plt.figure(figsize=(8, 4))

# Plotting the time series of given dataframe
plt.plot(df_new["time"], df_new[vis_var])

# Plotting the time series of given dataframe
plt.plot(df_old["time"], df_old[vis_var])

# Giving title to the chart using plt.title
plt.title("input dfs")

# rotating the x-axis tick labels at 30degree
# towards right
plt.xticks(rotation=30, ha="right")

# Providing x and y label to the chart
plt.xlabel("time")
plt.ylabel(vis_var)

Plot the output dataset

In [ ]:
# Create a figure with a specific size
plt.figure(figsize=(8, 4))

# Plotting the time series of given dataframe
plt.plot(df_concat["time"], df_concat[vis_var])

# Giving title to the chart using plt.title
plt.title("concatenated df")

# rotating the x-axis tick labels at 30degree
# towards right
plt.xticks(rotation=30, ha="right")

# Providing x and y label to the chart
plt.xlabel("time")
plt.ylabel(vis_var)